# The curse of dimensionality


Here is a function that depends on time $t$ and 3 other parameters. 

In this example class we generate many samples of the function and 
then try to build an interpolator that approximates this function as accurately as possible.

You can think of this function as a time series that depends on paramaters $a,b,c$.

In this problem class, we will assume the ranges of the parameters are:

- $0<a<1$

- $-0.5<b<0.5$

- $5<c<10$


When not varied we will fix the values to $a=0.1$, $b=-0.13$, $c=9$.


We will always assume time $t$ is between 0 and 1. Our time grid is `t = np.linspace(0, 1, 100)`, unless otherwise requested.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.interpolate import griddata
from pyDOE import *
from ipywidgets import interactive
import ipywidgets as widgets

In [ ]:
def f(t, a, b, c):
    return np.sqrt(a) * np.exp(-b * t) * np.sin(c * t) + 0.5 * np.cos(2 * t)

# Example usage
# Define parameters
a = 0.1
b = -0.13
c = 9


# Define time range
t = np.linspace(0, 1, 100)  # time from 0 to 1 with 100 points

# Calculate f(t) for these parameters
f_values = f(t, a, b, c)

# Plot the function
plt.figure(figsize=(10, 6))
plt.plot(t, f_values, label=r'$f(t; a, b, c)$', color='blue')
plt.xlabel('Time $t$')
plt.ylabel(r'$f(t)$')
plt.title(r'Time Series $f(t; a, b, c)$')
plt.legend()
plt.grid(True)
plt.show()


You can make a widget plot and play with different parameter values.  

In [ ]:
def plot_with_parameters(a, b, c):
    # Define time range
    t = np.linspace(0, 1, 100)
    
    # Calculate f(t) for these parameters 
    f_values = f(t, a, b, c)
    
    # Plot the function
    plt.figure(figsize=(10, 6))
    plt.plot(t, f_values, label=r'$f(t; a, b, c)$', color='blue')
    plt.xlabel('Time $t$')
    plt.ylabel(r'$f(t)$')
    plt.title(r'Time Series $f(t; a, b, c)$')
    plt.legend()
    plt.grid(True)
    plt.show()

# Create interactive widget
interactive_plot = interactive(
    plot_with_parameters,
    a=widgets.FloatSlider(min=0, max=1, step=0.01, value=0.1),
    b=widgets.FloatSlider(min=-0.5, max=0.5, step=0.01, value=-0.13),
    c=widgets.FloatSlider(min=5, max=10, step=0.1, value=9)
)

display(interactive_plot)

The goal of this problem class is to sample this function across all its parameter space and create interpolators. 

You should aim to understand the limitations of interpolation and how to deal with this very important problem. 

## Question 1

Consider all parameter values (except $t$) are fixed and create an interpolator with respect to time $t$. 

You use the original grid for the interpolation. 

In [ ]:
t = np.linspace(0, 1, 100)

### Question 1 Answer

In [ ]:
# Define function
def f(t, a, b, c):
    return np.sqrt(a) * np.exp(-b * t) * np.sin(c * t) + 0.5 * np.cos(2 * t)

# Define parameters
a = 0.1
b = -0.13
c = 9

# Calculate f(t) for these parameters
f_values = f(t, a, b, c)

# Def an interpolating function
g = interp1d(t, f_values, kind='linear')

## Question 2

Evaluate the interpolator on a much finer grid than the original $t$ grid. 

In [ ]:
# Finer time grid for evaluation
t_fine = np.linspace(0, 1, 1000)
f_interp = g(t_fine)
f_exact=f(t_fine, a, b, c)

## Question 3

Show the results of question 2 in a plot, showing both the exact function and the predictions of the interpolator. 

In [ ]:
# Plot the original samples and the interpolated function
plt.figure(figsize=(10, 6))
plt.plot(t, f_values, 'o', label='Original Samples', color='black', markersize=5)
plt.plot(t_fine, f_interp, '-', label='Interpolated Curve', color='blue', linewidth=3)
plt.plot(t_fine, f_exact, '-', label='Exact Function', color='red', linewidth=1.5)
plt.xlabel('Time $t$')
plt.ylabel(r'$f(t)$')
plt.title('Original Samples and Interpolated Function')
plt.legend()
plt.grid(True)
plt.show()

## Question 4

Show the ratio of the interpolated values to true values accross the fine time grid. 

What do you observe? Does it make sense?


In [ ]:
ratio = f_interp / f_exact

plt.figure(figsize=(10, 6))
plt.plot(t_fine, ratio, label='Ratio of Interpolated to Exact', color='purple')
plt.plot(t_fine, f_exact, '-', label='Exact Function', color='blue', linewidth=1.5)
plt.xlabel('Time $t$')
plt.ylabel('Ratio/'r'$f(t)$')
plt.title('Ratio of Interpolated Values to True Values')
plt.legend()
plt.grid(True)
plt.show()

## Question 5

Consider now all paramaters fixed except $a$ (and $t$). 

We assume the parameter $a$ can take values between 0 and 1.

Generate 10 samples of $f$ (i.e., 10 time series)  corresponding to linearly spaced values of $a$ spanning the interval.

Store them in a `pandas` DataFrame and plot them with the `plot` method of the DataFrame. 


In [ ]:
# Define function
def f(t, a, b, c):
    return np.sqrt(a) * np.exp(-b * t) * np.sin(c * t) + 0.5 * np.cos(2 * t)

# Define parameters
a = np.linspace(0,1,10)
b = -0.13
c = 9
t = np.linspace(0, 1, 100)

# Create an empty DataFrame to store the time series data
df = pd.DataFrame(index=t)

# Compute and store the time series for each a value
for i in a:
    f_values = f(t, i, b, c)
    df[f'a={i:.2f}'] = f_values 

print(df)

In [ ]:
# Plotting
plt.figure(figsize=(12, 8))
for column in df.columns:
    plt.plot(t, df[column], label=column)
plt.xlabel('Time $t$')
plt.ylabel(r'$f(t)$')
plt.title('Time Series for Different Values of $a$')
plt.legend()
plt.grid(True)
plt.show()

## Question 6 

Create an interpolator that interpolates over $a$ (same range as previous question) and returns the full time series (i.e., values of $f$ for all time points) over the original time grid, i.e., $t$. 

In [ ]:
t = np.linspace(0, 1, 100)

In [ ]:
# Def an interpolating function
g = interp1d(a, df.values, kind='linear', axis=1) # Axis = 1 means the interpolation is conducted over columns of the df (ie. for each value of a)

Plot the result for $a=0.125$, like in  question 3.

Plot the ratio, like in question 4. 

In [ ]:
a_test = 0.125
f_exact = f(t, 0.125, b, c)
f_interp = g(a_test)

print(f_interp)
print(f_exact)

plt.figure(figsize=(12, 6))
plt.plot(t, f_interp, label=f'Interpolated Series for a={a_test:.3f}',color='blue', linewidth=1)
plt.plot(t, f_exact, label=f'Exact Series for a={a_test:.3f}', color='red', linewidth=1)
plt.xlabel('Time $t$')
plt.ylabel(r'$f(t)$')
plt.title(f'Interpolated Time Series for a={a_test:.3f}')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
ratio = f_interp / f_exact

plt.figure(figsize=(10, 6))
plt.plot(t, ratio, label='Ratio of Interpolated to Exact', color='purple')
plt.plot(t, f_exact, '-', label='Exact Function', color='blue', linewidth=1.5)
plt.xlabel('Time $t$')
plt.ylabel('Ratio/'r'$f(t)$')
plt.title(f'Ratio of Interpolated Values to True Values when a = {a_test}')
plt.legend()
plt.grid(True)
plt.show()

## Question 7

Use `widgets` from `ipywidgets` to create a sliding scale of `a` values. 

In this question, you should plot the ratio between the interpolated values and the true values of the function evaluated at the original time grid $t$. 




In [ ]:
# Define function
def f(t, a, b, c):
    return np.sqrt(a) * np.exp(-b * t) * np.sin(c * t) + 0.5 * np.cos(2 * t)

# Define interactive plotting function
def inter_plot(a_test):
    a = np.linspace(0,1,10)
    t = np.linspace(0,1,100)
    b = -0.13
    c = 9

    # Create an empty DataFrame to store the time series data
    df = pd.DataFrame(index=t)

    # Compute and store the time series for each a value
    for i in a:
        f_values = f(t, i, b, c)
        df[f'a={i:.2f}'] = f_values 

    # Def an interpolating function
    g = interp1d(a, df.values, kind='linear', axis=1)
    
    f_exact = f(t, a_test, b, c)
    f_interp = g(a_test)

    ratio = f_interp / f_exact

    plt.figure(figsize=(10, 6))
    plt.plot(t, ratio, label='Ratio of Interpolated to Exact', color='purple')
    plt.plot(t, f_exact, '-', label='Exact Function', color='blue', linewidth=1.5)
    plt.xlabel('Time $t$')
    plt.ylabel('Ratio/'r'$f(t)$')
    plt.title('Ratio of Interpolated Values to True Values')
    plt.legend()
    plt.grid(True)
    plt.show()

interactive_plott = interactive(
    inter_plot,
    a_test=widgets.FloatSlider(min=0, max=1, step=0.01, value=0.1)
)

display(interactive_plott)

## Question 8

From your results of Question 7, what do you observe? Does it make sense?


## Question 9 

We will now consider both $a$ and $b$ as interpolation parameters. 


Our interpolator should therefore interpolate accross both $a$ and $b$ ranges. 

Generate $10^2$ parameter value pairs $(a,b)$ in the range $0<a<1$ and $-0.5<b<0.5$ using [latin hyper cube](https://pythonhosted.org/pyDOE/randomized.html) sampling. 


Show the $a$ and $b$ samples as a 2D scatter plot. 


In [ ]:
# Obtaining 10 random values of a and b which are seeded and comply with range
np.random.seed(5)

a=lhs(1,samples=10).flatten()
#print(a)

b_unscaled=lhs(1,samples=10).flatten()
b = np.interp(b_unscaled, (0, 1), (-0.5, 0.5)) # To comply with range of b
#print(b)

# Defining a grid to print A and B
A, B = np.meshgrid(a, b)
#print("A.shape, B.shape:", A.shape, B.shape)

# Flatten the meshgrid arrays to use them as coordinates for the scatter plot
A_flat = A.flatten()
B_flat = B.flatten()

# Create the scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(A_flat, B_flat, color='blue', marker='o')
plt.xlabel('Parameter $a$')
plt.ylabel('Parameter $b$')
plt.title('Scatter Plot of $(a, b)$ Values')
plt.grid(True)
plt.show()

## Question 10

For comparison, on the same plot, add a uniformly sampled realization of $10^2$ $a$ and $b$ values.


In [ ]:
aa = np.linspace(0,1,10)
#print(a)

bb = np.linspace(-0.5,0.5,10)
#print(b)

# Defining a grid to print A and B
AA, BB = np.meshgrid(aa, bb)
#print("A.shape, B.shape:", A.shape, B.shape)

# Flatten the meshgrid arrays to use them as coordinates for the scatter plot
AA_flat = AA.flatten()
BB_flat = BB.flatten()

# Create the scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(A_flat, B_flat, label='LHS Sample', color='blue', marker='o')
plt.scatter(AA_flat, BB_flat, label='Uniformly Spaced', color='red', marker='x')
plt.xlabel('Parameter $a$')
plt.ylabel('Parameter $b$')
plt.legend()
plt.title('Scatter Plot of $(a, b)$ Values')
plt.grid(True)
plt.show()

Can you distinguish by eye? 

As an extension question, think of how you would proceed if 
you needed to prove that these samples come from different generative processes (uniform or Latin Hyper Cube). 

## Question 11


Create the interpolator over the parameter space $(a,b)$, interpolating over samples of the function evaluated at the original time grid $t$.

As you will realise, we are dealing with an irregular grid and need the `griddata` method ([doc](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.griddata.html)).  

In [ ]:
# Define function
def f(t, a, b, c):
    return np.sqrt(a) * np.exp(-b * t) * np.sin(c * t) + 0.5 * np.cos(2 * t)

# Defining parameters
c = 9
t = np.linspace(0, 1, 100)

# Irregularly spaced data points
# Obtaining 10 random values of a and b which are seeded and comply with range
np.random.seed(5)
a_sample = lhs(1, samples=10).flatten()
b_sample = np.interp(lhs(1, samples=10).flatten(), (0, 1), (-0.5, 0.5))
#print(a_sample, b_sample)

# Create an empty DataFrame to store the time series data
sample_df = pd.DataFrame(index=t)

# Compute and store the time series for each a value
for i, j in zip(a_sample, b_sample):
    f_values = f(t, i, j, c)
    sample_df[f'a={i:.2f}, b={j:.2f}'] = f_values 

#print(sample_df)
#print(sample_df.size) # 100x10

In [ ]:
# Define a grid where you want to interpolate
ai = np.linspace(0, 1, 10)
bi = np.linspace(-0.5, 0.5, 10)
a_grid, b_grid = np.meshgrid(ai, bi)

# Empty DataFrame to store interpolated data
interp_df = pd.DataFrame(index=pd.MultiIndex.from_product([ai, bi], names=['a', 'b']))
print(interp_df)

# Interpolate
for i, j in enumerate(t):
    z_values = sample_df.iloc[i].values
    f_interp = griddata((a_sample, b_sample), z_values, (a_grid, b_grid), method='cubic')
    interp_df[f't={j:.2f}'] = f_interp.flatten()

print(interp_df)

In [ ]:
# Define a function to plot the interpolation for a given time step
def interp_plot1(t=0.5):
    # Find the index of the time step closest to the chosen value of `t`
    t_idx = (np.abs(t - sample_df.index)).argmin()
    z_values = sample_df.iloc[t_idx].values  # Extract z-values for this time step
    
    # Perform interpolation for this time step
    f_interp = griddata((a_sample, b_sample), z_values, (a_grid, b_grid), method='cubic')
    
    # Plot the interpolated grid
    plt.figure(figsize=(8, 6))
    plt.pcolormesh(ai, bi, f_interp, shading='auto')
    plt.scatter(a_sample, b_sample, c=z_values, edgecolor='k', label='Sample Points')  # Show original points
    plt.colorbar(label='Interpolated Values')
    plt.title(f"Interpolation for Time t={t:.2f}")
    plt.xlabel('a')
    plt.ylabel('b')
    plt.legend()
    plt.show()

# Create an interactive widget
interactive_plot = interactive(
    interp_plot1,
    t=widgets.FloatSlider(min=0, max=1, step=0.01, value=0.5)
)

display(interactive_plot)

interactive(children=(FloatSlider(value=0.5, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

## Question 12 

Show results with a sliding scale plot for $a$ and $b$.

On a different sliding bar plot, show the ratio between the interpolated values and the true values of the function evaluated at the original time grid $t$. 

What do you observe? 

Does it make sense? 

## Question 13

Compare memory and time of (i) the original function call and (ii) the interpolator call. 

Comment. 

## Question 14

Add a third parameter, $c$ to our interpolation problem and repeat questions 9 to 13. 

For this parameter, use a range of $5<c<10$. 

Comment on the results. 

## Question 15

Instead of interpolating with the griddata method, use (i) a gaussian process, and (ii) a neural network whose output layer is the function predicted at the $10^2$ points of the time grid and input layer is a,b,c values.  

What do you observe in terms of (i) time and (ii) memory? Comment on scalability in all cases covered. 


**Tips**: For the neural net part of this question use Google Colab and the GPUs there. Training should take a few minutes. A good idea for you to practice is also to use CSD3 to solve this question.
